In [1]:
from ROOT import TH1F, TFile, gROOT, gDirectory, TCanvas
import math
import numpy as np
import matplotlib.pyplot as plt

Welcome to JupyROOT 6.24/06


In [2]:
# Function to rescale the cross-section from 7 TeV to 13 TeV
# Taken from milliq_mcgen/blob/master/oniaDirect/Atlas_7_TeV/makeOutput.py
# Made it work for float or numpy ptrans 
def LHCBRescale(flag, ptrans):
    qt = ptrans
    if hasattr(qt , '__len__'):
        np.clip(qt, -1, 25, out=qt)
    else:
        qt = 25

    if flag == 1:   # upsilon 1S
        return 1.152 + 0.05074*qt - 0.0007649*qt*qt
    elif flag == 2: # upsilon 2S
        return 1.07 + 0.05387*qt - 0.001126*qt*qt
    elif flag == 3: # upsilon 3S
        return 1.192 + 0.0516*qt - 0.001176*qt*qt

    print("Illegal call to LHCBRescale")
    return 1.0

In [3]:
inFiles  = ["ups1S_combined.root",      "ups2S_combined.root",      "ups3S_combined.root"]
outFiles = ["ups1S_combined_run3.root", "ups2S_combined_run3.root", "ups3S_combined_run3.root"]
for i in [0,1,2]:
    
    fin = TFile(inFiles[i])
    # Get the contents of the histograms and the bin edges and centers
    # Remove underflow
    central    = np.array(gROOT.FindObject("central"))[1:]
    # print(central)
    up         = np.array(gROOT.FindObject("up"))[1:]
    down       = np.array(gROOT.FindObject("down"))[1:]
    bins       = np.array(gROOT.FindObject("central").GetXaxis().GetXbins())
    binsCenter = 0.5 * (bins[:-1]+bins[1:])

    # the scale factor to go from 13 to 13.6 extrapolated from the scale factor to go from 7 to 13
    scale = 1 + ((LHCBRescale(i+1, binsCenter) - 1) * (13.6-13)/(13-7)) 
    # add the overflow bin
    scale = np.append(scale, scale[-1])
    
    # scale the arrays
    central = scale*central
    up      = scale*up
    down    = scale*down
    
    # we need this later
    y1 = gROOT.FindObject("central")
    
    # open a new file
    newfile = TFile(outFiles[i], "RECREATE")
    
    # The central, up, and down output histograms
    hcen  = y1.Clone()
    hup   = y1.Clone()
    hdown = y1.Clone()
    hcen.SetName("central")
    hcen.SetTitle("central")
    hup.SetName("up")
    hup.SetTitle("up")
    hdown.SetName("down")
    hdown.SetTitle("down")
    hcen.SetYTitle("BR(mumu) * dsigma/dpt (nb/GeV) for abs(y)<1.2")
    hup.SetYTitle("BR(mumu) * dsigma/dpt (nb/GeV) for abs(y)<1.2")
    hdown.SetYTitle("BR(mumu) * dsigma/dpt (nb/GeV) for abs(y)<1.2")
    
    # zero everything out
    hcen.Reset("ICESM")
    hup.Reset("ICESM")
    hdown.Reset("ICESM")
    
    # fill the histograms
    for i in range(len(central)):
        j = i + 1
        hcen.SetBinContent(j, central[i])
        hup.SetBinContent(j, up[i])
        hdown.SetBinContent(j, down[i])
        hcen.SetBinError(j, 0)
        hup.SetBinError(j, 0)
        hdown.SetBinError(j, 0)
    
    # Write and close files
    newfile.Write()
    newfile.Close()
    fin.Close()

[9.89275649e-02 2.82306969e-01 4.32378948e-01 5.41470528e-01
 6.26428187e-01 7.14591920e-01 7.61765361e-01 7.69992411e-01
 7.46654510e-01 6.90009594e-01 6.20195210e-01 6.11965239e-01
 5.43294549e-01 4.78983492e-01 4.35642987e-01 3.91575009e-01
 3.42589259e-01 3.02617520e-01 2.68798172e-01 2.35263556e-01
 1.88689113e-01 1.40552461e-01 1.10913709e-01 8.25574398e-02
 6.75682724e-02 5.19322380e-02 4.06774879e-02 3.18038240e-02
 2.50272918e-02 1.98803898e-02 1.34880000e-02 8.83200000e-03
 5.92800000e-03 4.15200000e-03 2.88000000e-03 2.05200000e-03
 1.52640000e-03 1.13280000e-03 7.96800000e-04 6.02400000e-04
 4.89600000e-04 3.07200000e-04 1.93680000e-04 1.23840000e-04
 7.99200000e-05 3.88800000e-05 1.03680000e-05 1.55520000e-06
 0.00000000e+00]
[3.71086225e-02 1.02433927e-01 1.26169890e-01 1.60220966e-01
 1.57900378e-01 1.32499054e-01 1.16647534e-01 1.00338414e-01
 7.85734877e-02 6.76861182e-02 4.61983122e-02 3.01062446e-02
 2.08340865e-02 1.31870387e-02 8.38985853e-03 5.71200000e-03
 3.7680

In [4]:
central

array([1.79800566e-02, 4.37642713e-02, 6.73473458e-02, 6.86008824e-02,
       6.97408209e-02, 6.61485622e-02, 6.43623837e-02, 5.60234576e-02,
       4.75355328e-02, 3.74338951e-02, 2.68321937e-02, 2.02942797e-02,
       1.37954420e-02, 8.87224368e-03, 6.01811546e-03, 4.13068186e-03,
       2.91721596e-03, 2.03247264e-03, 1.49340312e-03, 1.02139488e-03,
       7.76363280e-04, 6.03551520e-04, 4.69428960e-04, 3.45623520e-04,
       3.06934320e-04, 1.89061224e-04, 1.18388952e-04, 8.35686720e-05,
       5.77758720e-05, 3.68837040e-05, 1.89835008e-05, 5.10697440e-06,
       5.93234400e-07, 0.00000000e+00])

In [5]:
len(central)

34

In [6]:
len(scale)

34